In [1]:
import os
from pathlib import Path

os.chdir(Path(os.getcwd()).parents[2])
os.getcwd()

target = "mth" # mth, pce

In [2]:
import torch
import torch.nn as nn
import numpy as np
import kaleido
from torch.utils.data import DataLoader
from data.perovskite_dataset import (
    PerovskiteDataset1d,
    PerovskiteDataset2d,
    PerovskiteDataset3d,
    PerovskiteDataset2d_time,
)
from models.resnet import ResNet152, ResNet, BasicBlock, Bottleneck
from models.slowfast import SlowFast
from data.augmentations.perov_2d import normalize as normalize_2d
from data.augmentations.perov_3d import normalize as normalize_3d
from base_model import seed_worker
import xgboost as xgb
from torchmetrics import MeanAbsoluteError, MeanSquaredError
from data.augmentations.perov_1d import normalize as normalize_1d
from argparse import ArgumentParser
from os.path import join

import dice_ml
from dice_ml import Dice

data_dir = "/home/l727n/Projects/Applied Projects/ml_perovskite/preprocessed"

if target == "pce":
    checkpoint_dir = "/home/l727n/E132-Projekte/Projects/Helmholtz_Imaging_ACVL/KIT-FZJ_2021_Perovskite/data_Jan_2022/checkpoints"

    path_to_checkpoint = join(
        checkpoint_dir, "1D-epoch=999-val_MAE=0.000-train_MAE=0.490.ckpt"
    )
else:
    checkpoint_dir = "/home/l727n/E132-Projekte/Projects/Helmholtz_Imaging_ACVL/KIT-FZJ_2021_Perovskite/data_Jan_2022/mT_checkpoints/checkpoints"

    path_to_checkpoint = join(
        checkpoint_dir, "mT_1D_RN152_full-epoch=999-val_MAE=0.000-train_MAE=40.332.ckpt"
    )


/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Import of model and computation of counterfactuals (higher and lower than response confidence intervall)

In [3]:
from data.augmentations.perov_1d import Normalize1D


val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print("Fold: ", fold_nb)

    train_mean, train_std = PerovskiteDataset1d(
    data_dir,
    transform=None,
    fold=fold_nb,
    scaler=None,
    no_border=False,
    return_unscaled= False if target == "pce" else True,
    label="PCE_mean" if target == "pce" else "meanThickness",
    ).get_stats()

    trainset = PerovskiteDataset1d(
    data_dir,
    transform=Normalize1D(train_mean, train_std),
    scaler=None,
    fold=fold_nb,
    no_border=False,
    return_unscaled= False if target == "pce" else True,
    label="PCE_mean" if target == "pce" else "meanThickness",
    split="train",
    val=False,
    )

    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)

    valset = PerovskiteDataset1d(
        data_dir=data_dir,
        transform=normalize_1d(train_mean, train_std),
        scaler=None,
        no_border=False,
        return_unscaled= False if target == "pce" else True,
        fold=fold_nb,
        split="train",
        label="PCE_mean" if target == "pce" else "meanThickness",
        val=True,
    )
    
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)

    for timeseries, label in trainloader:

        time = timeseries.numpy().reshape(-1, 4 * 719)  # [:,3,:]

        xgbr = xgb.XGBRegressor(
            n_estimators=100,
            tree_method="gpu_hist",
            n_jobs=20,
            max_depth=6,
            learning_rate=0.3 if target == "pce" else 0.15,
            booster="gbtree",
            num_parallel_tree=1,
            objective="reg:squarederror",
        )
        xgbr.fit(time, label.numpy())

    for timeseries, label in valloader:

        time = timeseries.numpy().reshape(-1, 4 * 719)  # [:,3,:]
        scores = xgbr.predict(time)

        mse = MeanSquaredError()
        mae = MeanAbsoluteError()

        val_mse = mse(torch.from_numpy(scores), label)
        print("MSE: ", val_mse)
        val_mse_folds.append(val_mse)

        val_mae = mae(torch.from_numpy(scores), label)
        print("MAE:", val_mae)
        val_mae_folds.append(val_mae)


print("Val Mean MSE: ", np.mean(val_mse_folds))
print("Val Mean MAE: ", np.mean(val_mae_folds))



Fold:  0
MSE:  tensor(8142.9771)
MAE: tensor(67.0261)
Fold:  1
MSE:  tensor(6574.0098)
MAE: tensor(59.6515)
Fold:  2
MSE:  tensor(10251.9971)
MAE: tensor(74.4183)
Fold:  3
MSE:  tensor(9825.9531)
MAE: tensor(74.6508)
Fold:  4
MSE:  tensor(12619.0820)
MAE: tensor(63.9992)
Val Mean MSE:  9482.804
Val Mean MAE:  67.94918


In [4]:
trainset_full = PerovskiteDataset1d(
    data_dir,
    transform=Normalize1D(train_mean, train_std),
    scaler=None,
    no_border=False,
    return_unscaled= False if target == "pce" else True,
    label="PCE_mean" if target == "pce" else "meanThickness",
)

trainloader_full = DataLoader(
    trainset_full, batch_size=len(trainset_full), shuffle=False
)

xgbr = xgb.XGBRegressor(
    n_estimators=100,
    tree_method="gpu_hist",
    n_jobs=20,
    max_depth=6,
    learning_rate=0.3 if target == "pce" else 0.15,
    booster="gbtree",
    num_parallel_tree=1,
    objective="reg:squarederror",
)

timeseries, label = next(iter(trainloader_full))

time = timeseries.numpy().reshape(-1, 4 * 719)  # [:,3,:]

xgbr.fit(time, label.numpy())

for timeseries, label in valloader:

    time = timeseries.numpy().reshape(-1, 4 * 719)  # [:,3,:]
    scores = xgbr.predict(time)

    mse = MeanSquaredError()
    mae = MeanAbsoluteError()

    val_mse = mse(torch.from_numpy(scores), label)
    print("MSE: ", val_mse)
    val_mse_folds.append(val_mse)

    val_mae = mae(torch.from_numpy(scores), label)
    print("MAE:", val_mae)
    val_mae_folds.append(val_mae)


MSE:  tensor(6.7554)
MAE: tensor(1.9470)


In [5]:
import pandas as pd
feature_important = xgbr.get_booster().get_score(importance_type='gain')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=True)
data = data.nlargest(10, columns="score")

In [7]:
import plotly.graph_objects as go
cd = ["#E1462C", "#0059A0", "#5F3893", "#FF8777","#0A2C6E", "#CEDEEB"]


fig = go.Figure()

loc = [0,1,2,4,7] if target == "pce" else [5,9]

fig.add_trace(go.Bar(
    y = data["score"].iloc[loc].iloc[::-1],
    x = np.array(["56", "106","101","617","14", "588","573","59","352","431"] if target == "pce" else  ["481","537","525","541","542","178","337","527","262","166"])[loc][::-1],
    marker_color= "#E3AF5F",
    name = "Early",
))

loc = [8,9] if target == "pce" else [0,6,8]

fig.add_trace(go.Bar(
    y = data["score"].iloc[loc].iloc[::-1],
    x = np.array(["56", "106","101","617","14", "588","573","59","352","431"] if target == "pce" else  ["481","537","525","541","542","178","337","527","262","166"])[loc][::-1],
    marker_color= cd[5],
    name = "Middle",
))

loc = [3,5,6] if target == "pce" else [1,2,3,4,7]

fig.add_trace(go.Bar(
    y = data["score"].iloc[loc].iloc[::-1],
    x = np.array(["56", "106","101","617","14", "588","573","59","352","431"] if target == "pce" else  ["481","537","525","541","542","178","337","527","262","166"])[loc][::-1],
    marker_color= cd[3],
    name = "Late",
))


fig.update_yaxes(title = "Feature Importance Score"#, range = [0,81]
)
fig.update_xaxes(title = "Timesteps", categoryarray= np.array(["14", "56","59","101","106", "352","431","573","588","617"] if target == "pce" else  ["166","178","262","337","481","525","527","537","541","542"]))

fig.update_layout(
    showlegend=True,
    margin=dict(pad = 5),
    template="plotly_white",
    height=400,
    width=500,
)

fig.write_image("xai/images/"+ target + "/1D/1D_tree.png", scale=2)

fig.show()

In [11]:
a1 = np.array(values[0:719])
a2 = np.array(values[719:1438])
a3 = np.array(values[1438:2157])
a4 = np.array(values[2157:719])

In [5]:
import pandas as pd

data = timeseries.numpy().reshape(-1, 4 * 719)
data = pd.DataFrame(data)

label = pd.DataFrame(label.numpy())
data["target"] = label

feature_names = data.columns[0:-1].to_list()


data_dice = dice_ml.Data(
    dataframe=data, continuous_features=feature_names, outcome_name="target"
)


In [6]:
import dice_ml
from dice_ml.utils import helpers

model_dice = dice_ml.Model(model=xgbr, backend="sklearn", model_type="regressor")

cf_methode = "genetic"
methode = Dice(data_dice, model_dice, method=cf_methode)


In [7]:
n = 0

query_instances = pd.DataFrame(data.iloc[n][0:-1]).T
genetic = methode.generate_counterfactuals(
    query_instances,
    total_CFs=1,
    desired_range=[0.9, 100] if target == "pce" else [1100, 5000],
)

cf_high = np.array(genetic.cf_examples_list[0].final_cfs_df)[:, 0:-1].reshape(4, 719)
x = np.array(data.iloc[n][0:-1]).reshape(4, 719)

if target == "pce":
    scaler = PerovskiteDataset1d(
        data_dir=data_dir,
        transform=normalize_1d(train_mean, train_std),
        fold=None,
        split="train",
        no_border=False,
        return_unscaled= False if target == "pce" else True,
        label="PCE_mean" if target == "pce" else "meanThickness",
        val=False,
        ).get_fitted_scaler()

    y = np.round(scaler.inverse_transform(data.iloc[n]["target"].reshape([-1, 1])), 2)
    y_cf_high = np.round(
        scaler.inverse_transform(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1])
        ),
        2,
    )
else:
    y = np.round(data.iloc[n]["target"].reshape([-1, 1]), 2)
    y_cf_high = np.round(
        np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1]), 2,
    )



100%|██████████| 1/1 [00:07<00:00,  7.40s/it]


In [8]:
genetic = methode.generate_counterfactuals(
    query_instances, total_CFs=1, desired_range=[-100, -0.9] if target == "pce" else [0, 600]
)

cf_low = np.array(genetic.cf_examples_list[0].final_cfs_df)[:, 0:-1].reshape(4, 719)

if target == "pce":
    y_cf_low = np.round(
        scaler.inverse_transform(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1])
        ),
        2,
    )
else:
    y_cf_low = np.round(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1]),
        2,
    )


100%|██████████| 1/1 [00:07<00:00,  7.14s/it]


#### Plot

In [23]:
import plotly.graph_objects as go


def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f"<b>{title}</b>"
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f"{title}<br>{subtitle}"


from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2,
    cols=4,
    subplot_titles=(
        format_title("High" if target == "pce" else "Thicker", "ND"),
        format_title("", "LP725"),
        format_title("", "LP780"),
        format_title("", "SP775"),
        format_title("Low" if target == "pce" else "Thinner"),
        None,
        None,
        None,
    ),
)

fig.add_trace(
    go.Scatter(y=x[0], name="ND", marker_color="grey", opacity=0.3, showlegend=False),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(y=cf_high[0], name="cf ND", marker_color="#042940", showlegend=False),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        y=x[1], name="LP725", marker_color="grey", opacity=0.3, showlegend=False
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(y=cf_high[1], name="cf LP725", marker_color="#005C53", showlegend=False),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        y=x[2], name="LP780", marker_color="grey", opacity=0.3, showlegend=False
    ),
    row=1,
    col=3,
)
fig.add_trace(
    go.Scatter(y=cf_high[2], name="cf LP780", marker_color="#9FC131", showlegend=False),
    row=1,
    col=3,
)
fig.add_trace(
    go.Scatter(
        y=x[3], name="SP775", marker_color="grey", opacity=0.3, showlegend=False
    ),
    row=1,
    col=4,
)
fig.add_trace(
    go.Scatter(
        y=cf_high[3], name="cf NSP775D", marker_color="#DBF227", showlegend=False
    ),
    row=1,
    col=4,
)

fig.add_trace(
    go.Scatter(y=x[0], name="ND", marker_color="grey", opacity=0.3, showlegend=False),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(y=cf_low[0], name="cf ND", marker_color="#042940", showlegend=False),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        y=x[1], name="LP725", marker_color="grey", opacity=0.3, showlegend=False
    ),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(y=cf_low[1], name="cf LP725", marker_color="#005C53", showlegend=False),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(
        y=x[2], name="LP780", marker_color="grey", opacity=0.3, showlegend=False
    ),
    row=2,
    col=3,
)
fig.add_trace(
    go.Scatter(y=cf_low[2], name="cf LP780", marker_color="#9FC131", showlegend=False),
    row=2,
    col=3,
)
fig.add_trace(
    go.Scatter(
        y=x[3], name="SP775", marker_color="grey", opacity=0.3, showlegend=False
    ),
    row=2,
    col=4,
)
fig.add_trace(
    go.Scatter(
        y=cf_low[3], name="cf NSP775D", marker_color="#DBF227", showlegend=False
    ),
    row=2,
    col=4,
)

fig.update_yaxes(title=None)
fig.update_yaxes(title_text=None)
fig.update_xaxes(title=None)

fig.update_yaxes(title="Intensity", row=1, col=1)
fig.update_yaxes(title="Intensity", row=2, col=1)
fig.update_xaxes(title="Timesteps", row=2, col=1)

if target == "pce":
    title = format_title(
        "Perovskite 1D Boosted Regression Tree Model",
        "Counterfactual Explanation ("
        + str(cf_methode)
        + ") / True PCE: "
        + str(*y[0])
        + " / CF PCE High: "
        + str(*y_cf_high[0])
        + " / CF PCE Low: "
        + str(*y_cf_low[0]),
    )
else:
    title = format_title(
        "Perovskite 1D Boosted Regression Tree Model",
        "Counterfactual Explanation ("
        + str(cf_methode)
        + ") / True mean Thickness (mTH): "
        + str(*y[0]) + "[nm]"
        + " / CF mTH Thick: "
        + str(*y_cf_high[0])
        + " / CF mTH Thin: "
        + str(*y_cf_low[0]),
    )

fig.update_layout(
    title=title,
    legend_title=None,
    title_y=0.95,
    title_x=0.035,
    template="plotly_white",
    height=400,
    width=1200,
)

#fig.write_image("xai/images/"+ target + "/1D/1D_cf.png", scale=2)

fig.show()


## PyTorch 1D Model

In [6]:
#### 1D Model

hypparams = {
    "dataset": "Perov_1d",
    "dims": 1,
    "bottleneck": False,
    "name": "ResNet152",
    "data_dir": data_dir,
    "no_border": False,
    "resnet_dropout": 0.0,
    "stochastic_depth": 0.0,
    "norm_target": True if target == "pce" else False,
    "target": "PCE_mean" if target == "pce" else "meanThickness"
}

model = ResNet.load_from_checkpoint(
    path_to_checkpoint,
    block=BasicBlock,
    num_blocks=[4, 13, 55, 4],
    num_classes=1,
    hypparams=hypparams,
)

model.eval()
print("Loaded")


tensor([0.2697, 0.0191, 0.0057, 0.0216]) tensor([0.1589, 0.0106, 0.0030, 0.0145])
Loaded


In [7]:
class Wrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, image):
        x = torch.Tensor(image.values)
        x = image.reshape(-1, 4, 719)
        y_pred = self.model(x)

        return y_pred.detach().numpy()

    def predict(self, image):
        x = torch.Tensor(image.values)
        x = x.reshape(-1, 4, 719)
        y_pred = self.model(x)
        y_pred = y_pred.detach().squeeze(1).numpy()

        return y_pred


In [8]:
import dice_ml
from dice_ml.utils import helpers

wModel = Wrapper(model)

model_dice = dice_ml.Model(model=wModel, backend="sklearn", model_type="regressor")

cf_methode = "genetic"
methode = Dice(data_dice, model_dice, method=cf_methode)


In [9]:
n = 0

query_instances = pd.DataFrame(data.iloc[n][0:-1]).T
genetic = methode.generate_counterfactuals(
    query_instances,
    total_CFs=1,
    desired_range=[0.9, 100] if target == "pce" else [1100, 5000],
)

cf_high = np.array(genetic.cf_examples_list[0].final_cfs_df)[:, 0:-1].reshape(4, 719)
x = np.array(data.iloc[n][0:-1]).reshape(4, 719)

if target == "pce":
    scaler = PerovskiteDataset1d(
        data_dir=data_dir,
        transform=normalize_1d(train_mean, train_std),
        fold=None,
        split="train",
        no_border=False,
        return_unscaled= False if target == "pce" else True,
        label="PCE_mean" if target == "pce" else "meanThickness",
        val=False,
        ).get_fitted_scaler()

    y = np.round(scaler.inverse_transform(data.iloc[n]["target"].reshape([-1, 1])), 2)
    y_cf_high = np.round(
        scaler.inverse_transform(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1])
        ),
        2,
    )
else:
    y = np.round(data.iloc[n]["target"].reshape([-1, 1]), 2)
    y_cf_high = np.round(
        np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1]), 2,
    )


100%|██████████| 1/1 [00:10<00:00, 10.14s/it]


In [10]:
genetic = methode.generate_counterfactuals(
    query_instances, total_CFs=1, desired_range=[-100, -0.9] if target == "pce" else [0, 600]
)

cf_low = np.array(genetic.cf_examples_list[0].final_cfs_df)[:, 0:-1].reshape(4, 719)

if target == "pce":
    y_cf_low = np.round(
        scaler.inverse_transform(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1])
        ),
        2,
    )
else:
    y_cf_low = np.round(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1]),
        2,
    )

100%|██████████| 1/1 [00:09<00:00,  9.25s/it]


In [11]:
import plotly.graph_objects as go


def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f"<b>{title}</b>"
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f"{title}<br>{subtitle}"

color = ["#E1462C", "#0059A0", "#5F3893", "#FF8777","#0A2C6E", "#CEDEEB"]

from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2,
    cols=4,
    subplot_titles=(
        format_title(" " if target == "pce" else " ", "ND"),
        format_title("", "LP725"),
        format_title("", "LP780"),
        format_title("", "SP775"),
        format_title(" " if target == "pce" else " "),
        None,
        None,
        None,
    ),
)

fig.add_trace(
    go.Scatter(y=x[0], name="ND", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(y=cf_high[0], name="cf ND", marker_color=color[1], showlegend=False, line=dict(width=2.5)),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        y=x[1], name="LP725", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(y=cf_high[1], name="cf LP725", marker_color=color[2], showlegend=False, line=dict(width=2.5)),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        y=x[2], name="LP780", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=1,
    col=3,
)
fig.add_trace(
    go.Scatter(y=cf_high[2], name="cf LP780", marker_color=color[0], showlegend=False, line=dict(width=2.5)),
    row=1,
    col=3,
)
fig.add_trace(
    go.Scatter(
        y=x[3], name="SP775", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=1,
    col=4,
)
fig.add_trace(
    go.Scatter(
        y=cf_high[3], name="cf NSP775D", marker_color=color[3], showlegend=False, line=dict(width=2.5)
    ),
    row=1,
    col=4,
)

fig.add_trace(
    go.Scatter(y=x[0], name="ND", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(y=cf_low[0], name="cf ND", marker_color=color[1], showlegend=False, line=dict(width=2.5)),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        y=x[1], name="LP725", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(y=cf_low[1], name="cf LP725", marker_color=color[2], showlegend=False, line=dict(width=2.5)),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(
        y=x[2], name="LP780", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=2,
    col=3,
)
fig.add_trace(
    go.Scatter(y=cf_low[2], name="cf LP780", marker_color=color[0], showlegend=False, line=dict(width=2.5)),
    row=2,
    col=3,
)
fig.add_trace(
    go.Scatter(
        y=x[3], name="SP775", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=2,
    col=4,
)
fig.add_trace(
    go.Scatter(
        y=cf_low[3], name="cf NSP775D", marker_color=color[3], showlegend=False, line=dict(width=2.5)
    ),
    row=2,
    col=4,
)

fig.update_yaxes(title=None,zeroline=False, showticklabels = False,)
fig.update_yaxes(title_text=None, range = [-2,5.2])
fig.update_xaxes(title=None, zeroline=False)

fig.update_yaxes(title="Intensity",showticklabels = True, row=1, col=1)
fig.update_yaxes(showticklabels = True, row=2, col=1)
fig.update_xaxes(title="Timesteps", row=2, col=1)

if target == "pce":
    title = format_title(
        "Perovskite 1D Boosted Regression Tree Model",
        "Counterfactual Explanation ("
        + str(cf_methode)
        + ") / True PCE: "
        + str(*y[0])
        + " / CF PCE High: "
        + str(*y_cf_high[0])
        + " / CF PCE Low: "
        + str(*y_cf_low[0]),
    )
else:
    title = format_title(
        "Perovskite 1D Boosted Regression Tree Model",
        "Counterfactual Explanation ("
        + str(cf_methode)
        + ") / True mean Thickness (mTH): "
        + str(*y[0]) + "[nm]"
        + " / CF mTH Thick: "
        + str(*y_cf_high[0])
        + " / CF mTH Thin: "
        + str(*y_cf_low[0]),
    )

fig.update_layout(
    title=title,
    legend_title=None,
    title_y=0.95,
    title_x=0.035,
    template="plotly_white",
    height=500,
    width=1200,
)

fig.write_image("xai/images/"+ target + "/1D/1D_cf_nn.png", scale=2)

fig.show()
